# Weather MCP Server Demo

This notebook demonstrates how to use the Weather MCP server to fetch weather data from Stormglass API.

## Features

- Get current weather conditions at any location
- Get weather forecasts (up to 240 hours)
- Get marine weather data (waves, swell, sea conditions)
- Get comprehensive weather data (combining standard + marine)
- Push notifications on successful weather queries

**Note**: Requires `STORMGLASS_API_KEY` in your `.env` file.


In [1]:
# Imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display

load_dotenv(override=True)


True

## Setup MCP Servers

We'll use two MCP servers:
1. **Weather Server** - For fetching weather data from Stormglass API
2. **Push Server** - For sending notifications on successful queries



In [2]:
import os
from pathlib import Path

current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Check if STORMGLASS_API_KEY is set
if not os.getenv("STORMGLASS_API_KEY"):
    print("⚠️ WARNING: STORMGLASS_API_KEY not found in environment variables.")
    print("Please set it in your .env file.")
else:
    print("✓ STORMGLASS_API_KEY found")

# Check if Pushover credentials are set
if not os.getenv("PUSHOVER_USER") or not os.getenv("PUSHOVER_TOKEN"):
    print("⚠️ WARNING: PUSHOVER_USER or PUSHOVER_TOKEN not found in environment variables.")
    print("Push notifications will not work. Please set them in your .env file.")
else:
    print("✓ Pushover credentials found")

weather_params = {
    "command": "uv",
    "args": ["run", "weather_server.py"]
}

push_params = {"command": "uv", "args": ["run", "push_server.py"]}

print("✓ MCP server parameters configured")


Current directory: /home/joshuaoluoch/projects/agents/6_mcp/community_contributions/weather_mcp
✓ STORMGLASS_API_KEY found
✓ Pushover credentials found
✓ MCP server parameters configured


## Test the Weather MCP Server

Let's first check what tools are available:


In [3]:
# List available tools from Weather MCP Server
async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as server:
    tools = await server.list_tools()
    print(f"Found {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool.name}: {tool.description[:100]}...")


Found 5 tools:
  - get_current_weather_tool: Get current weather conditions at a specific location.
    
    This tool fetches real-time weather ...
  - get_weather_forecast_tool: Get weather forecast for a specific location.
    
    This tool provides hourly weather forecasts i...
  - get_marine_weather_tool: Get marine weather data including wave height, swell, and sea conditions.
    
    This tool is part...
  - get_comprehensive_weather_tool: Get comprehensive weather data including both standard and marine weather.
    
    This tool combin...
  - geocode_location_tool: Geocode a location name to get its latitude and longitude coordinates.
    
    This tool converts l...


## Create a Weather Agent

This agent can fetch weather data and send notifications when queries are complete.


In [4]:
instructions = """
You are a weather assistant that helps users get weather information.
You have access to comprehensive weather data from Stormglass API including:
- Current weather conditions (temperature, humidity, wind, etc.)
- Weather forecasts (up to 240 hours)
- Marine weather data (waves, swell, sea conditions)
- Comprehensive weather data (combining standard + marine)

When users ask about weather:
1. Use get_current_weather_tool for current conditions
2. Use get_weather_forecast_tool for forecasts
3. Use get_marine_weather_tool for coastal/marine locations
4. Use get_comprehensive_weather_tool for complete weather picture

IMPORTANT:
- You can use EITHER location names (e.g., "Nairobi, Kenya") OR coordinates (latitude, longitude)
- If user provides a location name, use it directly - the tool will automatically geocode it
- If user provides coordinates, use them directly
- If both are provided, coordinates take precedence
- Present weather data in a clear, organized format
- Include units (Celsius for temperature, m/s for wind speed, etc.)
- After completing a successful weather query, you MUST call the 'push' tool to send a push notification
  with a summary of the weather information retrieved (e.g., "Weather for Nairobi: 14.4°C, 70% humidity, wind 1.55 m/s")

Be helpful and provide detailed weather information when requested.
"""


## Example 1: Get Current Weather (Using Location Name)

Get current weather conditions using just the location name - geocoding is automatic!


In [5]:
weather_query = "Get the current weather conditions for Nairobi, Kenya"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_current"):
            result = await Runner.run(agent, weather_query, max_turns=5)
            
        display(Markdown(result.final_output))


Here are the current weather conditions for **Nairobi, Kenya**:

- **Temperature**: 13.6°C
- **Humidity**: 96%
- **Wind Speed**: 2.71 m/s
- **Wind Direction**: 36.2° (Northeast)
- **Visibility**: 24.14 km
- **Cloud Cover**: 94%
- **Precipitation**: 0.0 mm

If you need further information, feel free to ask!

## Example 2: Get Weather Forecast (Using Location Name)

Get a weather forecast using just the location name.


In [6]:
forecast_query = "Get a 48-hour weather forecast for Mombasa, Kenya"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_forecast"):
            result = await Runner.run(agent, forecast_query, max_turns=5)
            
        display(Markdown(result.final_output))


Here's the 48-hour weather forecast for **Mombasa, Kenya**:

| Time (UTC)          | Temperature (°C) | Humidity (%) | Wind Speed (m/s) | Wind Direction (°) | Precipitation (mm) | Cloud Cover (%) |
|---------------------|-------------------|--------------|-------------------|-------------------|---------------------|------------------|
| 2025-11-10 08:00    | 25.87             | 44.93        | 2.68              | 85.54             | 0.05                | 24.27            |
| 2025-11-10 09:00    | 26.78             | 38.60        | 3.35              | 89.99             | 0.07                | 27.60            |
| 2025-11-10 10:00    | 26.91             | 40.33        | 3.40              | 95.47             | 0.05                | 22.97            |
| 2025-11-10 11:00    | 27.03             | 42.07        | 3.44              | 100.96            | 0.03                | 18.33            |
| 2025-11-10 12:00    | 27.16             | 43.80        | 3.49              | 106.44            | 0.00                | 13.70            |
| 2025-11-10 13:00    | 27.00             | 50.90        | 3.45              | 103.34            | 0.00                | 10.80            |
| 2025-11-10 14:00    | 26.84             | 58.00        | 3.40              | 100.24            | 0.00                | 7.90             |
| ...                 | ...               | ...          | ...               | ...               | ...                 | ...              |
| 2025-11-11 21:00    | 26.71             | 94.10        | 2.52              | 84.03             | 0.00                | 54.10            |
| 2025-11-11 22:00    | 26.43             | 94.10        | 2.18              | 80.57             | 0.00                | 68.57            |
| 2025-11-11 23:00    | 26.14             | 94.10        | 1.85              | 77.11             | 0.00                | 83.03            |
| 2025-11-12 00:00    | 25.86             | 94.10        | 1.51              | 73.65             | 0.01                | 97.50            |
| 2025-11-12 01:00    | 25.45             | 94.00        | 1.70              | 39.20             | 0.01                | 66.67            |
| 2025-11-12 02:00    | 25.03             | 93.90        | 1.88              | 4.74              | 0.02                | 35.83            |
| 2025-11-12 03:00    | 24.61             | 93.80        | 2.07              | 330.29            | 0.02                | 5.00             |
| 2025-11-12 04:00    | 24.57             | 81.57        | 2.10              | 357.11            | 0.02                | 36.07            |
| ...                 | ...               | ...          | ...               | ...               | ...                 | ...              |
| 2025-11-12 08:00    | 26.56             | 49.30        | 3.36              | 66.55             | 0.09                | 98.00            |

- **Temperature** ranges from **24.05°C** to **27.59°C**.
- **Humidity** varies between **44.93%** and **96.4%**.
- **Wind speeds** are between **0.45 m/s** and **4.59 m/s**.

A push notification with this summary has been sent successfully!

## Example 3: Get Marine Weather (Using Location Name)

Get marine weather data for a coastal location using just the location name.


In [7]:
marine_query = "Get marine weather data including wave height and swell conditions for Mombasa, Kenya for the next 24 hours"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_marine"):
            result = await Runner.run(agent, marine_query, max_turns=5)
            
        display(Markdown(result.final_output))


### Marine Weather Data for Mombasa, Kenya (Next 24 Hours)

#### Wave and Sea Conditions:
- **Maximum Wave Height**: 0.73 m
- **Minimum Wave Height**: 0.66 m
- **Wave Period**: 3.79 - 5.56 seconds
- **Wave Direction**: 100.89° to 104.82°

#### Swell Conditions:
- **Maximum Swell Height**: 0.58 m
- **Minimum Swell Height**: 0.55 m
- **Swell Period**: 6.76 - 7.31 seconds
- **Swell Direction**: 115.09° to 121.77°

#### Sea Level and Water Temperature:
- **Sea Level**: 0.42 - 0.43 m
- **Water Temperature**: 26.96 - 27.3 °C

---

A summary notification has been sent regarding the weather conditions. If you need more details or further assistance, feel free to ask!

## Example 4: Using Coordinates Directly

You can still use coordinates directly if you prefer or have them available.


In [8]:
# Example: Using coordinates directly (still works!)
coords_query = "Get current weather for latitude 1.2921, longitude 36.8219"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_coordinates"):
            result = await Runner.run(agent, coords_query, max_turns=5)
            
        display(Markdown(result.final_output))


Here's the current weather for the coordinates (1.2921, 36.8219):

- **Temperature:** 14.42°C
- **Humidity:** 70%
- **Wind Speed:** 1.55 m/s
- **Wind Direction:** 171.73°
- **Pressure:** 825.65 hPa
- **Visibility:** 24.14 km
- **Cloud Cover:** 100%
- **Precipitation:** 0.0 mm

A push notification has been sent with this weather information.

## Notes

- **Geocoding is automatic!** You can use location names like "Nairobi, Kenya" or "New York, USA"
- The tools will automatically convert location names to coordinates using Open-Meteo's free geocoding API
- You can still use coordinates directly if you prefer
- Make sure you have `STORMGLASS_API_KEY` set in your `.env` file
- Make sure you have `PUSHOVER_USER` and `PUSHOVER_TOKEN` set for push notifications
- Forecasts can be requested for up to 240 hours (10 days)
- Marine weather data is particularly useful for coastal locations
- Check your Pushover app for notifications about completed queries

## Check Traces

View detailed traces of the agent's weather queries:

https://platform.openai.com/traces
